In [ ]:
%pip install librosa

In [ ]:
import pandas as pd
import librosa
import numpy as np
import os

In [62]:
genres = [
    "gaming",
    "true_crime",
    "business",
    "science",
    "self_help",
    "comedy",
    "sports",
]

n_samples = 51
data = np.zeros((35, 2 + 2*n_samples), dtype=object)
data_idx = 0

for genre in genres:
    file_names = os.listdir("data/" + genre + "/")
    print(f"started {genre}")

    for file_name in file_names:
        y, sr = librosa.load("data/" + genre + "/" + file_name)
        pitches, _ = librosa.piptrack(y=y, sr=sr)

        data[data_idx, 0] = genre
        data[data_idx, 1] = file_name

        nonzero_mask = pitches != 0
        # average the pitches over time
        column_averages = np.nanmean(
            np.where(nonzero_mask, pitches, np.nan), axis=0
        )
        pitches_processed = np.nan_to_num(column_averages, nan=0)

        # grab the pitches at 0, 5, 10, ..., 100 percent
        final_pitches = []
        n = len(pitches_processed) - 1
        for idx in np.linspace(0, len(pitches_processed)-1, num=n_samples, endpoint=True, dtype=int):
            # data is in the form [pitch, percent] where percent is the time in the audio
            final_pitches.append(pitches_processed[idx])

        # file.write(genre + ", " + file_name + ", " + str(final_pitches) + "\n")

        final_pitches = np.array(final_pitches)
        data[data_idx, 2:2+n_samples] = final_pitches

        volume = librosa.feature.rms(y=y)

        # grab the volumes at 0, 5, 10, ..., 100 percent
        final_volumes = []
        n = volume.shape[1] - 1
        for idx in np.linspace(0, n, num=n_samples, endpoint=True, dtype=int):
            # data is in the form [volume, percent] where percent is the time in the audio
            final_volumes.append(np.sqrt(volume[0][idx]))

        # file.write(genre + ", " + file_name + ", " + str(final_volumes) + "\n")
        final_volumes = np.array(final_volumes)
        data[data_idx, 2+n_samples:] = final_volumes

        data_idx += 1
        # totals += final_volumes[:]

started gaming


/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_78844/1060650389.py:28: RuntimeWarning: Mean of empty slice
  column_averages = np.nanmean(


started true_crime
started business
started science
started self_help
started comedy
started sports


In [ ]:
def calc_volume(file):
    genres = [
        "gaming",
        "true_crime",
        "business",
        "science",
        "self_help",
        "comedy",
        "sports",
    ]

    for genre in genres:
        file_names = os.listdir("data/" + genre + "/")
        totals = np.zeros(21)

        for file_name in file_names:
            y, _ = librosa.load("data/" + genre + "/" + file_name)
            volume = librosa.feature.rms(y=y)

            # grab the volumes at 0, 5, 10, ..., 100 percent
            final_volumes = []
            n = volume.shape[1] - 1
            for percent in np.linspace():
                ind = int(np.floor(n * (percent / 100)))
                # data is in the form [volume, percent] where percent is the time in the audio
                final_volumes.append(np.sqrt(volume[0][ind]))

            file.write(genre + ", " + file_name + ", " + str(final_volumes) + "\n")

            final_volumes = np.array(final_volumes)
            totals += final_volumes[:]

        # calculate the average volume for the current genre
        totals = totals / 5
        i = 0
        averages_percents = []
        for percent in range(0, 105, 5):
            averages_percents.append(totals[i])
            i += 1

    return

In [63]:
columns = ['genre', 'name']
for metric in ['pitch', 'volume']:
  for p in range(n_samples):
    columns.append(f"{metric} - {p}")

# assert len(columns) == 44

In [69]:
pitch_cols = [f"pitch - {p}" for p in range(n_samples)]
vol_cols = [f"volume - {p}" for p in range(n_samples)]
print(pitch_cols)

['pitch - 0', 'pitch - 1', 'pitch - 2', 'pitch - 3', 'pitch - 4', 'pitch - 5', 'pitch - 6', 'pitch - 7', 'pitch - 8', 'pitch - 9', 'pitch - 10', 'pitch - 11', 'pitch - 12', 'pitch - 13', 'pitch - 14', 'pitch - 15', 'pitch - 16', 'pitch - 17', 'pitch - 18', 'pitch - 19', 'pitch - 20', 'pitch - 21', 'pitch - 22', 'pitch - 23', 'pitch - 24', 'pitch - 25', 'pitch - 26', 'pitch - 27', 'pitch - 28', 'pitch - 29', 'pitch - 30', 'pitch - 31', 'pitch - 32', 'pitch - 33', 'pitch - 34', 'pitch - 35', 'pitch - 36', 'pitch - 37', 'pitch - 38', 'pitch - 39', 'pitch - 40', 'pitch - 41', 'pitch - 42', 'pitch - 43', 'pitch - 44', 'pitch - 45', 'pitch - 46', 'pitch - 47', 'pitch - 48', 'pitch - 49', 'pitch - 50']


In [65]:
import json
df = pd.DataFrame(data, columns=columns)
pitch_data = df.drop(columns=vol_cols)
vol_data = df.drop(columns=pitch_cols)
genre_pitch = {}
genre_volume = {}
for genre in genres:
    genre_pitch[genre] = (
        pitch_data[pitch_data["genre"] == genre].mean(numeric_only=None).to_list()
    )
    genre_volume[genre] = (
        vol_data[vol_data["genre"] == genre].mean(numeric_only=None).to_list()
    )

/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_78844/2022912262.py:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pitch_data[pitch_data["genre"] == genre].mean(numeric_only=None).to_list()
/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_78844/2022912262.py:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  vol_data[vol_data["genre"] == genre].mean(numeric_only=None).to_list()
/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_78844/2022912262.py:9: FutureWarning: The default value of numeric_only in

In [57]:
pitch_data.mean()

/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_78844/2989391071.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pitch_data.mean()


pitch - 0     1145.386956
pitch - 1      979.090860
pitch - 2     1018.740645
pitch - 3      927.032307
pitch - 4      911.471539
pitch - 5     1154.535863
pitch - 6      999.754600
pitch - 7      922.336602
pitch - 8      986.450627
pitch - 9      918.390175
pitch - 10     851.754864
pitch - 11     868.527587
pitch - 12     899.894656
pitch - 13     981.664909
pitch - 14     909.392639
pitch - 15     902.884688
pitch - 16    1322.291000
pitch - 17    1045.731948
pitch - 18     955.072369
pitch - 19     996.747809
pitch - 20     870.849075
pitch - 21     984.483128
pitch - 22    1022.621569
pitch - 23     988.144830
pitch - 24    1078.758082
pitch - 25    1078.765855
pitch - 26     832.750082
pitch - 27    1010.213810
pitch - 28     970.366984
pitch - 29    1158.026572
pitch - 30    1140.836998
pitch - 31     791.822841
pitch - 32     943.657035
pitch - 33     768.306786
pitch - 34     977.713368
pitch - 35     797.173743
pitch - 36     993.746381
pitch - 37     804.638908
pitch - 38  

In [ ]:
vol_data.mean()

In [ ]:
all_data_tree = {}
for genre in genres:
    this_genre = {}
    for idx, row in df[df['genre'] == genre].iterrows():
        this_genre[row['name']] = {}
        this_genre[row["name"]]['pitch'] = row[pitch_cols].to_list()
        this_genre[row["name"]]['volume'] = row[vol_cols].to_list()
    all_data_tree[genre] = this_genre

In [ ]:
with open("./processed_data/aggregate.json", "w") as file:
  json.dump(all_data_tree, file)

In [ ]:
averages = {"pitch": genre_pitch, "volume": genre_volume}
with open("./processed_data/averages.json", "w") as file:
    json.dump(averages, file)

In [ ]:
df[pitch_cols].max(axis=None).min()

In [ ]:
y, sr = librosa.load("data/gaming/The Beginning Begins (Ep. 1) ｜ Fantasy High.mp3")
pitches, _ = librosa.piptrack(y=y, sr=sr)


In [ ]:
nonzero_mask = pitches != 0
        # average the pitches over time
column_averages = np.nanmean(
    np.where(nonzero_mask, pitches, np.nan), axis=0
)
pitches_processed = np.nan_to_num(column_averages, nan=0)

print(pitches_processed.shape)

In [ ]:
all_data_tree

In [72]:
pretty = {
    "gaming": "Gaming",
    "true_crime": "True Crime",
    "business": "Business",
    "science": "Science",
    "self_help": "Self Help",
    "comedy": "Comedy",
    "sports": "Sports",
}

avg_data_rep_2 = {"pitch": [], "volume": []}
for i in range(n_samples):
    pitches = {
        "idx": np.round(i / (n_samples-1) * 100),
    }
    volumes = {
        "idx": np.round(i / (n_samples-1) * 100),
    }
    volumes.update({f"{pretty[genre]}": genre_volume[genre][i] for genre in genres})
    pitches.update({f"{pretty[genre]}": genre_pitch[genre][i] for genre in genres})
    avg_data_rep_2["pitch"].append(pitches)
    avg_data_rep_2["volume"].append(volumes)

print(avg_data_rep_2["volume"])

[{'idx': 0.0, 'Gaming': 0.031202074897998866, 'True Crime': 0.11015136215668235, 'Business': 0.028696064834366552, 'Science': 0.026996922492980958, 'Self Help': 0.07255590041168034, 'Comedy': 0.010517213057705277, 'Sports': 0.1807528890669346}, {'idx': 2.0, 'Gaming': 0.21227202713489532, 'True Crime': 0.2172313928604126, 'Business': 0.23828174024820328, 'Science': 0.27620120346546173, 'Self Help': 0.24615356586873532, 'Comedy': 0.16985687837004662, 'Sports': 0.289948832988739}, {'idx': 4.0, 'Gaming': 0.33014230728149413, 'True Crime': 0.2184616044163704, 'Business': 0.21899539716541766, 'Science': 0.20611687637865544, 'Self Help': 0.13536803387105464, 'Comedy': 0.2929304838180542, 'Sports': 0.2742325752973557}, {'idx': 6.0, 'Gaming': 0.21382632851600647, 'True Crime': 0.1722441114485264, 'Business': 0.28545978367328645, 'Science': 0.19310585409402847, 'Self Help': 0.1349431797862053, 'Comedy': 0.24187261573970317, 'Sports': 0.16203061789274215}, {'idx': 8.0, 'Gaming': 0.195492097735404

In [ ]:
avg_data_rep_2[5]

In [ ]:
genre_pitch = {}
genre_volume = {}
for genre in genres:
    genre_pitch[genre] = (
        pitch_data[pitch_data["genre"] == genre].mean(numeric_only=None).to_list()
    )
    genre_volume[genre] = (
        vol_data[vol_data["genre"] == genre].mean(numeric_only=None).to_list()
    )

In [84]:
all_data = {}
for genre in genres:
    relevant = df[df['genre'] == genre]
    genre_data = {"pitch": [], "volume": []}
    for i in range(n_samples):
        pitches = {
          "idx": np.round(i / (n_samples - 1) * 100),
      }
        volumes = {
          "idx": np.round(i / (n_samples - 1) * 100),
      }
        volumes.update(
            {ep["name"][0:-4]: ep[f"volume - {i}"] for _, ep in relevant.iterrows()}
        )
        pitches.update(
            {ep["name"][0:-4]: ep[f"pitch - {i}"] for _, ep in relevant.iterrows()}
        )
        genre_data["pitch"].append(pitches)
        genre_data["volume"].append(volumes)
    all_data[genre] = genre_data

print(all_data)

{'gaming': {'pitch': [{'idx': 0.0, 'Dungeons and Daddies - S1E01 - A Man and His Handshake': 0.0, 'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Episode 1': 0.0, 'The Beginning Begins (Ep. 1) ｜ Fantasy High': 1860.350830078125, 'Comedy - Ep.1： Here There Be Gerblins - Chapter One': 2251.61669921875, 'Not Another D&D Podcast： Episode 1 - Green Teens Gone (The Moonstone Saga)': 0.0}, {'idx': 2.0, 'Dungeons and Daddies - S1E01 - A Man and His Handshake': 2155.892822265625, 'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Episode 1': 733.0584716796875, 'The Beginning Begins (Ep. 1) ｜ Fantasy High': 1384.0179443359375, 'Comedy - Ep.1： Here There Be Gerblins - Chapter One': 2023.7987060546875, 'Not Another D&D Podcast： Episode 1 - Green Teens Gone (The Moonstone Saga)': 947.0676879882812}, {'idx': 4.0, 'Dungeons and Daddies - S1E01 - A Man and His Handshake': 372.0957946777344, 'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Episode 1': 1121.121337890625, 'The Beginning Begins (Ep. 

In [85]:
with open("./src/data/all_data_lists.json", "w") as f:
    json.dump(all_data, f)

In [73]:
with open("./src/data/averages_list.json", "w") as f:
  json.dump(avg_data_rep_2, f) 

In [82]:
name_map = {}
for genre in genres:
    name_map[genre] = [name[0:-4] for name in df[df["genre"] == genre]['name'].to_list()]

with open("./src/data/name_lists.json", "w") as f:
    json.dump(name_map, f)

In [83]:
all_data

{'gaming': {'pitch': [{'idx': 0.0,
    'Dungeons and Daddies - S1E01 - A Man and His Handshake': 0.0,
    'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Episode 1': 0.0,
    'The Beginning Begins (Ep. 1) ｜ Fantasy High': 1860.350830078125,
    'Comedy - Ep.1： Here There Be Gerblins - Chapter One': 2251.61669921875,
    'Not Another D&D Podcast： Episode 1 - Green Teens Gone (The Moonstone Saga)': 0.0},
   {'idx': 2.0,
    'Dungeons and Daddies - S1E01 - A Man and His Handshake': 2155.892822265625,
    'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Episode 1': 733.0584716796875,
    'The Beginning Begins (Ep. 1) ｜ Fantasy High': 1384.0179443359375,
    'Comedy - Ep.1： Here There Be Gerblins - Chapter One': 2023.7987060546875,
    'Not Another D&D Podcast： Episode 1 - Green Teens Gone (The Moonstone Saga)': 947.0676879882812},
   {'idx': 4.0,
    'Dungeons and Daddies - S1E01 - A Man and His Handshake': 372.0957946777344,
    'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Epis

{'gaming': ['Dungeons and Daddies - S1E01 - A Man and His Handshake.mp3',
  'The Draw of Destiny ｜ Critical Role ｜ Campaign 3, Episode 1.mp3',
  'The Beginning Begins (Ep. 1) ｜ Fantasy High.mp3',
  'Comedy - Ep.1： Here There Be Gerblins - Chapter One.mp3',
  'Not Another D&D Podcast： Episode 1 - Green Teens Gone (The Moonstone Saga).mp3'],
 'true_crime': ["The Gitchie Manitou MURDERS： America's MOST HAUNTED Crime Scene ｜ Full Podcast ｜ MURDER IN AMERICA.mp3",
  'MrBallen Podcast ｜ Episode ＂4 Incredibly Strange Disappearances ＂ (PODCAST EPISODE).mp3',
  'The Itaewon Halloween Tragedy - 159 Dead & How The Police Tried To Cover It Up.mp3',
  'Gerard John Schaefer ＂The Hangman＂ ｜ Episode 358 ｜ Morbid： A True Crime Podcast.mp3',
  'Unhinged & Barbaric： The HORRIFYING Story of The Taylor Parker Case in Texas ｜ Reagan Hancock.mp3'],
 'business': ['The Ramsey Show (REPLAY).mp3',
  'Mark Zuckerberg Takes On Apple Fanboys, Tech Layoffs, Raising Cattle & More.mp3',
  'Andrew Tate EXCLUSIVE INTERV